<a href="https://colab.research.google.com/github/Sidra2499/ETL_Pipeline_for_Real_Time_Data_Processing/blob/main/ETL_Pipeline_for_Real_Time_Data_Processing_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!pip install --quiet requests
!pip install --quiet gspread
!pip install --quiet google-auth

In [24]:
import requests  # For extracting data from an API
import pandas as pd  # For data transformation
import gspread  # For loading data to Google Sheets
from google.colab import auth
from google.auth import default
import time
from gspread_dataframe import set_with_dataframe



In [25]:
import gspread
from google.oauth2.service_account import Credentials


In [26]:
# Define the scope for Google Sheets API access
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]

# Load credentials from the uploaded file (adjust path if necessary)
creds = Credentials.from_service_account_file('/content/realtimeetlpipeline-85be1c42ec0c.json', scopes=scope)
# Authorize gspread with the credentials
client = gspread.authorize(creds)


In [27]:
# Open your spreadsheet (Make sure the name matches EXACTLY as in Google Sheets)
spreadsheet = client.open("ETL_Pipeline")  # Replace with your actual Google Sheet name

# Select the first sheet
sheet = spreadsheet.sheet1

print("✅ Google Sheet connected successfully!")

✅ Google Sheet connected successfully!


In [ ]:
# Define API URL
api_url = "https://api.coingecko.com/api/v3/coins/markets"

# Define API URL
api_url = "https://api.coingecko.com/api/v3/coins/markets"

# Define parameters for the API request
params = {
    "vs_currency": "usd",  # Get prices in USD
    "order": "market_cap_desc",  # Order by market cap
    "per_page": 40,  # Limit to 40 coins
    "page": 1,
    "sparkline": "false"  # Don't include sparkline data
}

headers = {
    "Accept": "application/json",
    "User-Agent": "Mozilla/5.0"
}

while True:
    # Fetch new data from API
    response = requests.get(api_url, params=params, headers=headers)

    if response.status_code == 200:
        data = response.json()
        df = pd.DataFrame(data)

        # Clear old data (optional)
        sheet.clear()

        # Write new data to Google Sheets
        set_with_dataframe(sheet, df)

        print("✅ Data updated successfully!")

    else:
        print(f"⚠️ Failed to fetch data: {response.status_code}")

    # Wait for 5 minutes before fetching again
    time.sleep(300)


✅ Data updated successfully!
✅ Data updated successfully!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [33]:
# Convert API JSON response to Pandas DataFrame
df = pd.DataFrame(data, columns=["id", "symbol", "name", "current_price", "market_cap", "total_volume"])

# Display the first few rows
print(df.head())

            id symbol      name  current_price     market_cap  total_volume
0      bitcoin    btc   Bitcoin   88585.000000  1747316425040   71618347652
1     ethereum    eth  Ethereum    2394.930000   286773142472   40925303634
2       tether   usdt    Tether       0.999352   142270525923   87899087945
3       ripple    xrp       XRP       2.160000   123654963277   10736539782
4  binancecoin    bnb       BNB     604.140000    87866505056    1817860154


In [ ]:
from gspread_dataframe import set_with_dataframe

# Clear previous data (optional)
sheet.clear()

# Write the DataFrame to the Google Sheet
set_with_dataframe(sheet, df)

print("✅ Data successfully written to Google Sheets!")